# User workspace

Developing ML apps in shared JupyterLab environment running on Kubeflow + Kubernetes is an attractive option for various reasons relating to both cloud vs. local and JupyterLab vs. vim/etc., e.g.:
- Hardware accelerators not available on user's local machine can be attached as needed
- Users can share a common NFS space for TensorBoard logs and large datasets

## Development workflow

The workflow is approximately this:
- Initially models are debugged locally, in the JupyterLab environment; we can check that our model trains and perhaps achieves good performance on a single task, a single hparam set, and a single node.
- We can then train models as TFJobs to perform a range of tasks, explore hparam sets, or distribute training or rollouts over a large number of nodes.
    - TFJobs are Kubernetes CRD's and require model code to be encapsulated in a container; there are multiple options for preparing such a container from the code and dependencies in a user's workspace:
        - Use a remote container builder tool, e.g.
            - Google Container Builder
            - Argo
        - Attach a Docker daemon sidecar to the workspace environment and use Docker
        - Build in the workspace environment with a Docker alternative, preferably without requiring root at runtime, e.g.
            - buildah
            - ftl
        - Don't build a new container
            - simply stage user's workspace to GCS or NFS and when trainer runs have the first step be to stage in and set up (e.g. pip install -r requirements.txt)

Here we take a look at the setup and usage of a couple rootless container build options.

### buildah

https://github.com/projectatomic/buildah/blob/master/install.md

In [ ]:
!sudo apt-get update
!sudo apt-get -y upgrade
!sudo apt-get install -y make gcc git pkg-config
!sudo apt-get -y install golang-1.8

!sudo apt-get -y install software-properties-common
!sudo add-apt-repository -y ppa:alexlarsson/flatpak
!sudo add-apt-repository -y ppa:gophers/archive
!sudo apt-add-repository -y ppa:projectatomic/ppa
!sudo apt-get -y -qq update
!sudo apt-get install -y skopeo
!sudo apt-get -y install bats btrfs-tools git libapparmor-dev libdevmapper-dev libglib2.0-dev libgpgme11-dev libostree-dev libseccomp-dev libselinux1-dev skopeo-containers go-md2man

!git clone https://github.com/projectatomic/buildah $GOPATH/src/github.com/projectatomic/buildah
!cd $GOPATH/src/github.com/projectatomic/buildah
!make buildah imgtype TAGS="apparmor seccomp"
!sudo install -D -m0755 buildah /usr/local/bin/buildah

In [1]:
!buildah

NAME:
   buildah - an image builder

USAGE:
   buildah [global options] command [command options] [arguments...]

VERSION:
   0.15 (image-spec 1.0.0, runtime-spec 1.0.0)

COMMANDS:
     add                          Add content to the container
     build-using-dockerfile, bud  Build an image using instructions in a Dockerfile
     commit                       Create an image from a working container
     config                       Update image configuration settings
     containers                   List working containers and their base images
     copy                         Copy content into the container
     from                         Create a working container based on an image
     images                       List images in local storage
     inspect                      Inspects the configuration of a container or image
     mount                        Mount a working container's root filesystem
     push                         Push an image to a specified destination
 

In [4]:
!buildah images

mkdir /var/run/containers/storage: permission denied
mkdir /var/run/containers/storage: permission denied


In [15]:
!sudo buildah images
# None to show

ERRO[0000] 'overlay' is not supported over overlayfs    


In [58]:
import os
!mkdir ~/buildah-dev
os.chdir("/home/jovyan/buildah-dev")
!echo "FROM gcr.io/tensorflow/tensorflow:1.4.1" > Dockerfile
!sudo buildah bud --tag something .

mkdir: cannot create directory ‘/home/jovyan/buildah-dev’: File exists
ERRO[0000] 'overlay' is not supported over overlayfs    
STEP 1: FROM gcr.io/tensorflow/tensorflow:1.4.1
Getting image source signatures
Copying blob sha256:054be6183d067af1af06196d7123f7dd0b67f7157a0959bd857ad73046c3be9a
 44.40 MiB / 44.40 MiB [====================================================] 2s
Copying blob sha256:779578d7ea6e8cc3934791724d28c56bbfc8b1a99e26236e7bf53350ed839b98
 848 B / 848 B [============================================================] 0s
Copying blob sha256:82315138c8bd2f784643520005a8974552aaeaaf9ce365faea4e50554cf1bb44
 628 B / 628 B [============================================================] 0s
Copying blob sha256:88dc0000f5c4a5feee72bae2c1998412a4b06a36099da354f4f97bdc8f48d0ed
 860 B / 860 B [============================================================] 0s
Copying blob sha256:79f59e52a355a539af4e15ec0241dffaee6400ce5de828b372d06c625285fd77
 170 B / 170 B [===========================

In [59]:
!echo "FROM docker://tensorflow/tensorflow:1.4.1" > Dockerfile
!sudo buildah bud --tag something .

ERRO[0000] 'overlay' is not supported over overlayfs    
STEP 1: FROM docker://tensorflow/tensorflow:1.4.1
Getting image source signatures
Copying blob sha256:1be7f2b886e89a58e59c4e685fcc5905a26ddef3201f290b96f1eff7d778e122
 40.88 MiB / 40.88 MiB [====================================================] 2s
Copying blob sha256:6fbc4a21b806838b63b774b338c6ad19d696a9e655f50b4e358cc4006c3baa79
 846 B / 846 B [============================================================] 0s
Copying blob sha256:c71a6f8e13782fed125f2247931c3eb20cc0e6428a5d79edb546f1f1405f0e49
 620 B / 620 B [============================================================] 0s
Copying blob sha256:4be3072e5a37392e32f632bb234c0b461ff5675ab7e362afad6359fbd36884af
 854 B / 854 B [============================================================] 0s
Copying blob sha256:06c6d2f5970057aef3aef6da60f0fde280db1c077f0cd88ca33ec1a70a9c7b58
 171 B / 171 B [============================================================] 0s
Copying blob sha256:ed8e33f2ff8

In [60]:
!sudo buildah images

ERRO[0000] 'overlay' is not supported over overlayfs    


In [61]:
!sudo buildah containers

ERRO[0000] 'overlay' is not supported over overlayfs    
CONTAINER ID  BUILDER  IMAGE ID     IMAGE NAME                       CONTAINER NAME


### ftl

https://github.com/GoogleCloudPlatform/runtimes-common/tree/master/ftl/python

In [33]:
!bazel run //ftl:python_builder -- --help








Target //ftl:python_builder up-to-date:

  bazel-bin/ftl/python_builder






Traceback (most recent call last):
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/ftl/python_builder.runfiles/__main__/ftl/python/main.py", line 19, in <module>
    from containerregistry.tools import patched
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/ftl/python_builder.runfiles/containerregistry/tools/__init__.py", line 19, in <module>
    from containerregistry.tools import patched_
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/ftl/python_builder.runfiles/containerregistry/tools/patched_.py", line 23, in <module>
    import httplib2
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/ftl/py

In [34]:
# HACK
!mv /opt/conda/bin/python ~/bk/

In [35]:
import os
os.chdir("/home/jovyan/lib/runtimes-common")

In [36]:
!bazel run //ftl:python_builder -- --help








Target //ftl:python_builder up-to-date:

  bazel-bin/ftl/python_builder






usage: main.py [-h] (--base BASE | --tar_base_image_path TAR_BASE_IMAGE_PATH)
               --name NAME --directory DIRECTORY
               [--cache-repository CACHE_REPOSITORY] [--no-cache] [--cache]
               [--global-cache] [--no-upload] [--upload]
               [--output-path OUTPUT_PATH]
               [-v [{INFO,WARNING,CRITICAL,ERROR,DEBUG,NOTSET}]]
               [--destination DESTINATION_PATH] [--entrypoint ENTRYPOINT]
               [--exposed-ports EXPOSED_PORTS] [--python-cmd PYTHON_CMD]
               [--pip-cmd PIP_CMD] [--virtualenv-cmd VENV_CMD]

Construct python images from source.

optional arguments:
  -h, --help            show this help message and exit
  --base BASE           The name of the docker base image.
  --tar_base_image_path TAR_BASE_IMAGE_PATH
                        The tar path for the base dockerimage for the FTL
                        build
  --name NAME  

In [17]:
!bazel run //ftl:python_builder -- --name gcr.io/kubeflow-rl/test:latest \
        --directory /home/jovyan/ \
        --base gcr.io/cloud-builders/docker:latest








Target //ftl:python_builder up-to-date:

  bazel-bin/ftl/python_builder






2018-03-03,14:59:53.540 INFO     Beginning FTL build for python
2018-03-03,14:59:53.540 INFO     FTL arg passed: 
 exposed_ports None
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 cache_repository None
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 tar_base_image_path None
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 name gcr.io/kubeflow-rl/test:latest
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 pip_cmd pip
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 global_cache False
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 cache True
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 upload True
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 venv_cmd virtualenv
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 base gcr.io/cloud-builders/docker:latest
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 output_path None
2018-03-03,14:59:53.541 INFO     FTL arg passed: 
 direc

In [19]:
!sudo bazel run //ftl:python_builder -- --name gcr.io/kubeflow-rl/test:latest \
        --directory /home/jovyan/examples-new/agents \
        --base gcr.io/cloud-builders/docker:latest








Target //ftl:python_builder up-to-date:

  bazel-bin/ftl/python_builder






2018-03-03,15:03:56.429 INFO     Beginning FTL build for python
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 exposed_ports None
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 cache_repository None
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 tar_base_image_path None
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 name gcr.io/kubeflow-rl/test:latest
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 pip_cmd pip
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 global_cache False
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 cache True
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 upload True
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 venv_cmd virtualenv
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 base gcr.io/cloud-builders/docker:latest
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 output_path None
2018-03-03,15:03:56.429 INFO     FTL arg passed: 
 direc

In [ ]:
Looks like you need to use the docker client to authenticate with any repository requiring auth.

In [20]:
!sudo bazel run //ftl:python_builder -- --name quay.io/cwbeitel/test:latest \
        --directory /home/jovyan/examples-new/agents \
        --base gcr.io/cloud-builders/docker:latest








Target //ftl:python_builder up-to-date:

  bazel-bin/ftl/python_builder






2018-03-03,15:04:19.612 INFO     Beginning FTL build for python
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 exposed_ports None
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 cache_repository None
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 tar_base_image_path None
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 name quay.io/cwbeitel/test:latest
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 pip_cmd pip
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 global_cache False
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 cache True
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 upload True
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 venv_cmd virtualenv
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 base gcr.io/cloud-builders/docker:latest
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 output_path None
2018-03-03,15:04:19.612 INFO     FTL arg passed: 
 directo

In [31]:
# Instead of moving /opt/conda/bin/python (which is python 3+) can we specify 2.7
!sudo bazel run //ftl:python_builder -- --name quay.io/cwbeitel/test:latest \
        --directory /home/jovyan/dev \
        --base gcr.io/cloud-builders/docker:latest \
        --python-cmd python2








Target //ftl:python_builder up-to-date:

  bazel-bin/ftl/python_builder






2018-03-03,15:24:26.095 INFO     Beginning FTL build for python
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 exposed_ports None
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 cache_repository None
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 tar_base_image_path None
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 name quay.io/cwbeitel/test:latest
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 pip_cmd pip
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 global_cache False
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 cache True
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 upload True
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 venv_cmd virtualenv
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 base gcr.io/cloud-builders/docker:latest
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 output_path None
2018-03-03,15:24:26.095 INFO     FTL arg passed: 
 directo

In [ ]:
# So there seems to be an issue with the way we're using containerregistry...

In [42]:
!bazel run @containerregistry//:puller.par -- --help








Target @containerregistry//:puller.par up-to-date:

  bazel-bin/external/containerregistry/puller.par






usage: puller.par [-h] [--name NAME] [--directory DIRECTORY]
                  [--stderrthreshold STDERRTHRESHOLD]

Pull images from a Docker Registry, faaaaast.

optional arguments:
  -h, --help            show this help message and exit
  --name NAME           The name of the docker image to pull and save.
                        Supports fully-qualified tag or digest references.
  --directory DIRECTORY
                        Where to save the image's files.
  --stderrthreshold STDERRTHRESHOLD
                        Write log events at or above this level to stderr.


In [45]:
# Experiment with containerregistry tools
!bazel run @containerregistry//:puller.par -- --name gcr.io/cloud-builders/docker:latest --directory ~/dev








Target @containerregistry//:puller.par up-to-date:

  bazel-bin/external/containerregistry/puller.par








In [51]:
!bazel run @containerregistry//:pusher.par -- --name quay.io/cwbeitel/docker:latest --config /home/jovyan/dev/config.json








Target @containerregistry//:pusher.par up-to-date:

  bazel-bin/external/containerregistry/pusher.par






E0303 15:37:53.957439    6916 docker_session_.py:325] Error during upload of: quay.io/cwbeitel/docker:latest
Traceback (most recent call last):
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/external/containerregistry/pusher.par/__main__.py", line 154, in <module>
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/external/containerregistry/pusher.par/__main__.py", line 145, in main
  File "/home/jovyan/.cache/bazel/_bazel_jovyan/410acaca1fa83cbd688d55783b644966/execroot/__main__/bazel-out/k8-fastbuild/bin/external/container